In [2]:
!pip install pandas numpy lightgbm optuna scikit-learn
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import datetime as dt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.6 MB/s eta 0:00:00


In [9]:
train = pd.read_csv('train.csv', parse_dates=['doj'])
test = pd.read_csv('test.csv', parse_dates=['doj'])
transactions = pd.read_csv('transactions.csv', parse_dates=['doj', 'doi'])

In [10]:
trans_15 = transactions[transactions['dbd'] == 15].copy()

In [12]:
features = ['doj', 'srcid', 'destid', 'cumsum_seatcount', 'cumsum_searchcount',
            'srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']

trans_15 = trans_15[features]

train_merged = train.merge(trans_15, on=['doj', 'srcid', 'destid'], how='left')
test_merged = test.merge(trans_15, on=['doj', 'srcid', 'destid'], how='left')

In [13]:
def add_date_features(df):
    df['dayofweek'] = df['doj'].dt.dayofweek
    df['month'] = df['doj'].dt.month
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    return df

train_merged = add_date_features(train_merged)
test_merged = add_date_features(test_merged)

train_merged['search_to_book'] = train_merged['cumsum_searchcount'] / (train_merged['cumsum_seatcount'] + 1)
test_merged['search_to_book'] = test_merged['cumsum_searchcount'] / (test_merged['cumsum_seatcount'] + 1)

for col in ['cumsum_seatcount', 'cumsum_searchcount', 'search_to_book']:
    train_merged[col] = train_merged[col].fillna(0)
    test_merged[col] = test_merged[col].fillna(0)

cat_cols = ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    train_merged[col] = le.fit_transform(train_merged[col].astype(str))
    test_merged[col] = le.transform(test_merged[col].astype(str))
    le_dict[col] = le

In [14]:
features = ['srcid', 'destid', 'cumsum_seatcount', 'cumsum_searchcount', 'search_to_book',
            'srcid_region', 'destid_region', 'srcid_tier', 'destid_tier', 'dayofweek', 'month', 'is_weekend']

target = 'final_seatcount'
X = train_merged[features]
y = train_merged[target]
X_test = test_merged[features]

In [17]:
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100)
    }

    cv = GroupKFold(n_splits=5)
    oof_preds = np.zeros(X.shape[0])
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups=train_merged['doj'])):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        model = lgb.LGBMRegressor(**params, n_estimators=1000)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])

        oof_preds[val_idx] = model.predict(X_val, num_iteration=model.best_iteration_)

    rmse = np.sqrt(mean_squared_error(y, oof_preds))
    print(f"Trial RMSE: {rmse:.4f}")
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2025-06-18 13:46:32,726] A new study created in memory with name: no-name-5e3cdc0c-7832-472f-9263-1b5cd562d06a
[I 2025-06-18 13:46:50,850] Trial 0 finished with value: 453.4814659835137 and parameters: {'learning_rate': 0.13722548128787346, 'num_leaves': 84, 'feature_fraction': 0.7673518966985811, 'bagging_fraction': 0.7492962233045292, 'bagging_freq': 5, 'min_data_in_leaf': 75}. Best is trial 0 with value: 453.4814659835137.


Trial RMSE: 453.4815


[I 2025-06-18 13:47:17,524] Trial 1 finished with value: 460.5655367650035 and parameters: {'learning_rate': 0.10518138278412673, 'num_leaves': 228, 'feature_fraction': 0.8415538702561272, 'bagging_fraction': 0.8101408619500473, 'bagging_freq': 5, 'min_data_in_leaf': 90}. Best is trial 0 with value: 453.4814659835137.


Trial RMSE: 460.5655


[I 2025-06-18 13:48:01,255] Trial 2 finished with value: 446.2120118492755 and parameters: {'learning_rate': 0.05211071044675759, 'num_leaves': 34, 'feature_fraction': 0.7786371785582827, 'bagging_fraction': 0.8026477662267294, 'bagging_freq': 10, 'min_data_in_leaf': 83}. Best is trial 2 with value: 446.2120118492755.


...

Trial RMSE: 447.7635


[I 2025-06-18 14:03:39,976] Trial 28 finished with value: 446.85834590572136 and parameters: {'learning_rate': 0.07055702713133924, 'num_leaves': 128, 'feature_fraction': 0.8048577340774336, 'bagging_fraction': 0.8259916992935493, 'bagging_freq': 6, 'min_data_in_leaf': 34}. Best is trial 21 with value: 441.46127397810375.


Trial RMSE: 446.8583


[I 2025-06-18 14:04:10,347] Trial 29 finished with value: 441.7966215664142 and parameters: {'learning_rate': 0.04589505767842613, 'num_leaves': 92, 'feature_fraction': 0.7709512368818159, 'bagging_fraction': 0.790705369981963, 'bagging_freq': 3, 'min_data_in_leaf': 24}. Best is trial 21 with value: 441.46127397810375.


Trial RMSE: 441.7966


In [18]:
best_params = study.best_trial.params
best_params.update({"objective": "regression", "metric": "rmse", "verbosity": -1})

final_model = lgb.LGBMRegressor(**best_params, n_estimators=1000)
final_model.fit(X, y)

LGBMRegressor(bagging_fraction=0.8524166599608531, bagging_freq=8,
              feature_fraction=0.7381739363521873,
              learning_rate=0.03550693558050911, metric='rmse',
              min_data_in_leaf=22, n_estimators=1000, num_leaves=131,
              objective='regression', verbosity=-1)

In [19]:
test_preds = final_model.predict(X_test)

submission = pd.DataFrame({
    'route_key': test['route_key'],
    'final_seatcount': test_preds
})

submission.to_csv('submission_file.csv', index=False)
print("Submission saved as submission_file.csv")

Submission saved as submission_file.csv


In [21]:
oof_preds = final_model.predict(X)
total_rmse = np.sqrt(mean_squared_error(y, oof_preds))
print(f"Final RMSE on full training data: {total_rmse:.4f}")


Final RMSE on full training data: 307.2156


In [22]:
from google.colab import files
files.download('submission_file.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>